In [2]:
%reset
import numpy as np
import pandas as pd
import numpy.random as npr
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import scipy
import scipy.stats
from scipy.stats import norm

In [14]:
# Number of paths that need to be simulated
path = 1

# Number of timesteps for the simulator to create stock prices for
timesteps = 20

# Number of years that need to be simulated.
years_sim = 2

# The assumed volatility that is given.
volatility_asset = 0.22

# The start price of the stock.
start_price = 100

price_mov = 0.01

# The assumed risk free rate.
rf_rate = 0.00

# The seed number that is given, so that np.random will have the same random choice every run.
seed_number = 3011

# Reading/writing order in which we reshape the matrices
rw_order = 'F' # Order F, meaning that the first index with change fastest and last index changest slowest

In [19]:
'''Maximum Price after a number of timesteps'''
max_price = start_price*np.power((1+price_mov),timesteps)

'''Minimum Price after a number of timesteps'''
min_price = start_price*np.power((1-price_mov),timesteps)


In [20]:
min_price

81.79069375972307

In [37]:
def binomial_tree(X, ret, t, strike, prob, type):
    u = 1 + ret
    d = 1 - ret

    C = X * d ** (np.arange(t,-1,-1)) * u ** (np.arange(0,t+1,1))

    if type == "Call":
        C = np.maximum(C-strike, np.zeros(t+1))
    elif type == "Put":
        C = np.maximum(strike-C, np.zeros(t+1))


    for i in np.arange(t,0,-1):
        C = (prob*C[1:i+1]) + (1-prob) * C[0:i]

    return C[0]

In [38]:
binomial_tree(start_price, price_mov, timesteps, 120, 0.5, "Put")

20.000001925472258

In [ ]:
def alter_binomial_tree(X, ret, t, strike, prob, type):
    u = 1 + ret
    d = 1 - ret

    C = X * d ** (np.arange(t,-1,-1)) * u ** (np.arange(0,t+1,1))

    if type == "Call":
        C = np.maximum(C-strike, np.zeros(t+1))
    elif type == "Put":
        C = np.maximum(strike-C, np.zeros(t+1))


    for i in np.arange(t,0,-1):
        C = (prob*C[1:i+1]) + (1-prob) * C[0:i]

    return C[0]

In [ ]:
%%%%%%%%%% Problem and method parameters %%%%%%%%%%%%%
S = 5;E = 10;T = 1;r = 0.06;sigma = 0.3;M = 256;
dt = T/M;A = 0.5*(exp(-r*dt)+exp((r+sigmaˆ2)*dt));
u=A+ sqrt(Aˆ2-1);d = 1/u;p = (exp(r*dt)-d)/(u-d);
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% Option values at time T
W = max(E-S*d.ˆ([M:-1:0]’).*u.ˆ([0:M]’),0);
% log/cumsum version
csl = cumsum(log([1;[1:M]’]));
tmp = csl(M+1) - csl - csl(M+1:-1:1) + log(p)*([0:M]’) + log(1-p)*([M:-1:0]’);
value = exp(-r*T)*sum(exp(tmp).*W);
disp(’Option value is’), disp(value)

In [8]:
def stoch_pathsim(n, m, T, v, S, r):
    #Since, we need to set the seed before running each function, we put it inside the function.
    np.random.seed(seed_number)
    
    #We determine the delta of t: The number of simulation years divided by the number of time steps.
    delta_t = T/m
    delta_t = T
    
    #Then we determine the return + 1 of a stock price going up. Using the formula given in question 2.
    ret_up = 1 + price_mov

    #Then we determine the return + 1 of a stock price going down
    ret_down = 1/ret_up
    
    #Using the risk neutral probability, we determine the probability of the stock price going up (prob of ret_up happening)
    prob_up = 0.5 
    
    #We determine the probability of the stock price going down. Using the formula given in question 2.
    prob_down = 1-prob_up
    
    #We create a matrix (1 row) of all starting values and the length of the number of paths.
    row = 1
    starting_prices = np.full((row, n), S)
    
    #We create another matrix that is equal to size of the zero's in the price_slow matrix.
    #In this new matrix we store all the probabilities of the stock price going up an down. 

    #Using Numpy Random Choice, it will choose randomly (with the calculated probabilities) between two values, return up and down.
    probability = npr.choice((ret_down,ret_up), n*m, p=[prob_down,prob_up])
    
    #Then we reshape this list into a matrix and take the cumulative products over every column in the matrix.
    return_matrix = np.cumprod(np.reshape(probability, (m,n), order=rw_order),axis=0)
    
    #Now we have one matrix with all the cumulative products and one array with starting values.
    #If we multiply them, we have all the stock prices at every timestep and for every path.
    stock_prices = np.multiply((return_matrix),starting_prices)
    
    #We have a matrix of (m,n) without the initial starting values. Therefore, we concatenate these to the matrix.
    prices_stoch = np.concatenate((starting_prices, stock_prices), axis=0)
    
    #Return the matrix with all the prices.
    return prices_stoch

In [ ]:
def binomial_call(S0,T,m,rf,E,sigma):
    dt = T/m
    # Compute steps for tree: u and d
    u = 1 + ret
    d = 1 - ret
    # Compute the probability of up
    p = 0.5
    # Setup stock price and call value matrix
    # We need m+1 rows and columns due to the zero state!
    s = np.zeros((m+1,m+1))
    c = np.zeros((m+1,m+1))
    # Initialize stock price
    s[0,0] = S0
    
    # Compute all stock prices
    for i in range(1,m+1):
        # Loop over time periods
        for j in range(0,i+1):
            # Loop over states in a single time period
            if j == 0: # This is an up state
                s[j,i] = s[j,i-1]*u
            elif j > 0:
                s[j,i] = s[j-1,i-1]*d
    
    # Compute call value
    # We know the call value at the very last step
    c[:,m] = s[:,m] - E
    c[c < 0] = 0
    # We now reverse the order and go from the end back
    # to the beginning to compute the call value
    for i in range(m-1,-1,-1):
        for j in range(0,i+1):
                c[j,i] = (p*c[j,i+1]+(1-p)*c[j+1,i+1])*np.exp(-rf*dt)
    
    return c[0,0] # Return Discounted price

In [ ]:
def simulated_option(S0,T,m,rf,E,sigma,n):
    
    #First the simulated stock prices are being calculted using the same function as before.
    stock_prices = stoch_pathsim(n,m,T,sigma,S0,rf)
    
    #Then we extract the last row of the matrix, which are the final prices of all the stocks (paths).
    final_prices = stock_prices[-1,:]
    
    #We determine the call value by subtracting the strike price from the final prices.
    call_price = final_prices-E
    
    #We check werther the call value is lower than zero. If this is the case, then we set this value equal to zero,
    #because nobody will exercise his/her call when the value is negative. Thus the price = 0.
    call_price = call_price.clip(min=0)
    
    #These call_values are all at the last timestep. Meaning we should discount them to determine the present value.
    pv_price = call_price / np.exp(rf * T)
    
    #We return the mean of the present call value of all the stocks (number of paths).                                        
    return np.mean(pv_price)

In [ ]:
# Set the characteristics for all the three methods.
# Number of paths that need to be simulated
path = 1 #Number of paths is only used by the simulated call price

# Time to maturity
years = 1

# The assumed volatility of the underlying asset
volatility_asset = 0.01

# Starting price of the underlying asset.
start_price = 100

# The assumed risk free rate.
rf_rate = 0.00

# The strike price.
strike_price = 100

# The seed number that is given, so that np.random will have the same random choice every run.
seed_number = 3011

In [ ]:
def price_paths_fast(n, m, T, v, S, r):
    #Since, we need to set the seed before running each function, we put it inside the function.
    np.random.seed(seed_number)
    
    #We determine the delta of t: The number of simulation years divided by the number of time steps.
    delta_t = T/m
    
    #Then we determine the return + 1 of a stock price going up. Using the formula given in question 2.
    ret_up = np.exp(v*np.sqrt(delta_t))
    
    #Then we determine the return + 1 of a stock price going down
    ret_down = 1/ret_up
    
    #Using the risk neutral probability, we determine the probability of the stock price going up (prob of ret_up happening)
    prob_up = (np.exp(r*delta_t)-ret_down)/(ret_up-ret_down) #Using the formula given in question 2.
    
    #We determine the probability of the stock price going down. Using the formula given in question 2.
    prob_down = 1-prob_up
    
    #We create a matrix (1 row) of all starting values and the length of the number of paths.
    row = 1
    starting_prices = np.full((row, n), S)
    
    
    #We create another matrix that is equal to size of the zero's in the price_slow matrix.
    #In this new matrix we store all the probabilities of the stock price going up an down. 
    #Using Numpy Random Choice, it will choose randomly (with the calculated probabilities) between two values, return up and down.
    probability = npr.choice((ret_down,ret_up), n*m, p=[prob_down,prob_up])
    
    #Then we reshape this list into a matrix and take the cumulative products over every column in the matrix.
    return_matrix = np.cumprod(np.reshape(probability, (m,n), order=rw_order),axis=0)
    
    #Now we have one matrix with all the cumulative products and one array with starting values.
    #If we multiply them, we have all the stock prices at every timestep and for every path.
    stock_prices = np.multiply((return_matrix),starting_prices)
    
    #We have a matrix of (m,n) without the initial starting values. Therefore, we concatenate these to the matrix.
    prices_fast = np.concatenate((starting_prices, stock_prices), axis=0)
    
    #Return the matrix with all the prices.
    return prices_fast